In [ ]:
#  mnist识别
import torch 
import numpy as np
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
from torch import  nn
from torch.autograd import Variable
from torch import  optim
from torchvision import transforms
 
# 定义CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(
                nn.Conv2d(1,16,kernel_size=3), # 16, 26 ,26
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True))
        
        self.layer2 = nn.Sequential(
                nn.Conv2d(16,32,kernel_size=3),# 32, 24, 24
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2)) # 32, 12,12     (24-2) /2 +1
        
        self.layer3 = nn.Sequential(
                nn.Conv2d(32,64,kernel_size=3), # 64,10,10
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True))
        
        self.layer4 = nn.Sequential(
                nn.Conv2d(64,128,kernel_size=3),  # 128,8,8
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2))  # 128, 4,4
        
        self.fc = nn.Sequential(
                nn.Linear(128 * 4 * 4,1024),
                nn.ReLU(inplace=True),
                nn.Linear(1024,128),
                nn.ReLU(inplace=True),
                nn.Linear(128,10))
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x
 
 
# 预处理=>将各种预处理组合在一起
data_tf = transforms.Compose(
                [transforms.ToTensor(), # 将numpy.ndarray格式数据转换为torch.tensor格式
                 transforms.Normalize([0.5],[0.5])]) # 归一化处理，把0~255的灰度像素值数据归一化为均值为0.5、标准差为0.5的数据
  
# 使用内置函数下载mnist数据集
train_set = mnist.MNIST('./data',train=True,transform=data_tf,download=True)
test_set = mnist.MNIST('./data',train=False,transform=data_tf,download=True)
 
train_data = DataLoader(train_set,batch_size=64,shuffle=True)
test_data = DataLoader(test_set,batch_size=128,shuffle=False)
 
net = CNN()
criterion = nn.CrossEntropyLoss() # 使用交叉熵损失
optimizer = optim.SGD(net.parameters(),1e-1) # 随机梯度下降，学习率为0.1
 
nums_epoch = 20
 
# 开始训练
losses =[]
acces = []
eval_losses = []
eval_acces = []
 
for epoch in range(nums_epoch):
    print(epoch+1)
    train_loss = 0
    train_acc = 0
    net = net.train()
    for img , label in train_data:
        #img = img.reshape(img.size(0),-1) 
        img = Variable(img) # 转换成PyTorch的一个变量（新版本不需要）
        label = Variable(label)
        
        # 前向传播
        out = net(img)
        loss = criterion(out,label)
        # 反向传播
        optimizer.zero_grad() # 每次将梯度重置为0
        loss.backward() # 反向调整参数
        optimizer.step()
        
        # 记录误差
        train_loss += loss.item()
        # 计算分类的准确率
        _,pred = out.max(1) # 取评分最高的结果作为所分的类别
        num_correct = (pred == label).sum().item()
        acc = num_correct / img.shape[0]
       
        train_acc += acc
        
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    
    eval_loss = 0
    eval_acc = 0
    # 测试集不训练
    for img , label in test_data:
        #img = img.reshape(img.size(0),-1)
        img = Variable(img)
        label = Variable(label)
        
        out = net(img)
        
        loss = criterion(out,label)
        
        # 记录误差
        eval_loss += loss.item()
        
        _ , pred = out.max(1)
        num_correct = (pred==label).sum().item()
        acc = num_correct / img.shape[0]
        
        eval_acc += acc
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    
    print('Epoch {} Train Loss {} Train  Accuracy {} Teat Loss {} Test Accuracy {}'.format(
        epoch+1, train_loss / len(train_data),train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))
 


1
Epoch 1 Train Loss 0.12251093118043958 Train  Accuracy 0.9612206823027718 Teat Loss 0.04421999578473241 Test Accuracy 0.9849683544303798
2
Epoch 2 Train Loss 0.032869981871618735 Train  Accuracy 0.9896055437100213 Teat Loss 0.024924744118212817 Test Accuracy 0.991495253164557
3
Epoch 3 Train Loss 0.022034881753505202 Train  Accuracy 0.9934701492537313 Teat Loss 0.032236073035164736 Test Accuracy 0.990506329113924
4
Epoch 4 Train Loss 0.01611688208611392 Train  Accuracy 0.9948694029850746 Teat Loss 0.030782382982418116 Test Accuracy 0.9903085443037974
5
Epoch 5 Train Loss 0.012074834617083273 Train  Accuracy 0.996152052238806 Teat Loss 0.03324374901762606 Test Accuracy 0.9890229430379747
6
Epoch 6 Train Loss 0.009954630497956184 Train  Accuracy 0.9971515191897654 Teat Loss 0.024477557602166817 Test Accuracy 0.9926819620253164
7


In [1]:
pip install torch torchvision torchaudio -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/Note: you may need to restart the kernel to use updated packages.

